# Energy, the world, and the Smart Cities
## Exploring energy consumption and production patterns worldwide and in countries with top performing Smart Cities

Population in smart cities is growing, as well as Climate-related challenges. Smart cities are seen to become the new sustainable urban environment to tackle with these challenges with the help of digitalisation and big data analysis.

**In this part of the Smart Cities project** on Kaggle I am diving deeper into the energy consumption and production patterns worldwide, as well as in top performing Smart Cities. 

**Dataset source** 
1. **World Energy Consumption** 
Data on Energy by Our World in Data: this dataset is a collection of key metrics maintained by Our World in Data. It is updated regularly and includes data on energy consumption (primary energy, per capita, and growth rates), energy mix, electricity mix and other relevant metrics. This data has been collected, aggregated, and documented by Hannah Ritchie, Max Roser and Edouard Mathieu from different sources.
**Licence**
All visualizations, data, and code produced by Our World in Data are completely open access under the Creative Commons BY license. You have the permission to use, distribute, and reproduce these in any medium, provided the source and authors are credited.
[**Available here**](https://www.kaggle.com/datasets/pralabhpoudel/world-energy-consumption).

2. **Historical GHG emissions** 
Climate Watch: The PRIMAP-hist national historical emissions time series (1850-2019) Date Of Content: 1850-2019 Summary Of Licenses: Creative Commons Attribution-NonCommercial 4.0 International license Citation: Gütschow, J.; Günther, A.; Pflüger, M. (2021): The PRIMAP-hist national historical emissions time series v2.3.1 (1850-2019). 
[**Available here**](https://www.climatewatchdata.org/ghg-emissions).

[Check out the 1st part of the project here](https://www.kaggle.com/code/olgashevtsova/1-smart-cities-visualizing-sci-with-python?scriptVersionId=100552594).
[Check out the 2nd part of the project here.](https://www.kaggle.com/code/olgashevtsova/2-smart-cities-population-and-smi-raiting)

**Hashtags:** python, data cleaning, data exploration, vizualisation with Python.

**Topics:** Smart cities, Smart Cities Index, python

# Table of contents
* [1. Preparation and cleaning/formatting](#section-one)
* [2. World Energy Consumption: animation and visualization from 1990](#section-two)
    * [2.1 World Energy Consumption](#subsection-one)
    * [2.2 Energy Consumption profile in countries with the highest GHG emissions (per capita)](#subsection-two)
    * [2.3 Energy Consumption profiles for countries with top performing Smart Cities](#subsection-three)
    * [2.4 Historical Renewable and Low carbon energy consumption patterns](#subsection-four)
* [3. Energy Production patterns](#section-three)
   * [3.1 Fossil fuels energy production profiles for countries with the highest GHG emissions levels (per capita) from 1990](#subsection-one-one)
   * [3.2 Fossil fuels production of countries with top performing Smart Cities](#subsection-one-two)
* [4. Insights](#section-four)

In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import numpy as np
import scipy
import math
import collections

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

<a id="section-one"></a>
# 1. Preparation, cleaning and formatting

In [2]:
d=pd.read_csv('/kaggle/input/world-energy-consumption/World Energy Consumption.csv')
d.info()

In [3]:
d.describe()

# It is clear that data reqiored cleaning (some Inf, NaN values), as well as probably some outliers. Also the 'year' columns needs to be converted to integer. 

In [4]:
d.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
# Seems there are a lot of null values and replacement of some of them will lead to the fact that a lot of data will be missing, so let's replace Nulls with forward/backward filling.
# As data for the earliest years is missing so we will further focus on the most recent decades where more raw data is availabe

d=d.fillna(method='ffill').fillna(method='bfill')

In [6]:
#Convertinf float dates to integers

d['year']=pd.to_datetime(d['year'],format='%Y')
d['year']=d['year'].dt.year

In [7]:
# Added one more column with additional calculation

d['gdp_per_capita']=d['gdp']/d['population']

In [8]:
# Final check for Null and NaNs values:

d.isnull().sum()
nandf=d[d.isna().any(axis=1)]
nandf

In [9]:
d.notnull().sum()

In [10]:
# Looking at all countries list to check if there are regions in the list

a=[]
for c in d['country']:
    if c not in a:
        a.append(c)
    

In [11]:
# Assigning indexies for columns to furhter explore separately

d.columns
x=-1
for column in d.columns:
    x+=1
    print(x, column)

In [12]:
d.describe()

<a id="section-two"></a>
# 2. Energy Consumption

<a id="subsection-one"></a>
### 2.1 World Energy Consumption patterns

In [13]:
df=d.copy()

# Looking at the data with the highest/lowest changes (%) of reneables energy consumption.

filt=df['year']>1970
countriesgrouped=df[filt].groupby('country').get_group('World')

# Interestingly enough is that the renewbles/lowe carbon energy consumption is increasing throughout the last decade. 

countriesgrouped.nlargest(10, ['low_carbon_energy_per_capita','renewables_energy_per_capita'])

In [14]:
# The lower levels of general energy consumption significantly decreased during well-known global crisises like energy crisis in 1970-80s, in 2000s and mid-2010s and during pandemic as we will see later as well.

countriesgrouped.nsmallest(10, ['energy_cons_change_pct', 'fossil_cons_change_pct'])

In [15]:
# Let's have a look at the Primary energy consumption profiles for year 2000 and the latest available - 2020.

we=d.copy()
we=we.iloc[:, [1,2, 14, 21, 44, 52, 61, 70, 76, 83, 109, 117]]
we=we[(we['year'] >= 1990) & (we['year'] <= 2020)]

#Using median methon to avoid outliers.

we=we.groupby(by ='year', axis=0, as_index=False).median()

we.rename(columns = {'biofuel_share_energy': 'Biofuel', 'coal_share_energy': 'Coal', 'gas_share_energy': 'Gas', 'hydro_share_energy' : 'Hydro', 'nuclear_share_energy' : 'Nuclear', 'oil_share_energy' : 'Oil', 'solar_share_energy' :'Solar', 'wind_share_energy': 'Wind'}, inplace=True)

sources = ['Biofuel', 'Coal', 'Gas', 'Hydro', 'Nuclear', 'Oil', 'Solar', 'Wind']

we = pd.melt(we, id_vars='year', 
             value_vars=sources, 
             var_name='Energy source', 
             value_name='Share of primary energy consumption')

fig, axes = plt.subplots(1,2, figsize=(30, 15))

#Creating the first plot with 1990-2020 data

for i in sources:
    axes[0].plot(we[we['Energy source'] == i]['year'], 
            we[we['Energy source'] == i]['Share of primary energy consumption'], 
            alpha =0.7, 
            linewidth =1, fillstyle='full')

fig.legend()

year_start, year_end = 1990, 2020
names = we[['Energy source', 'Share of primary energy consumption']][we['year'] == year_end-1].sort_values(by='Share of primary energy consumption')

for i, value in names.itertuples(index=False):
    axes[0].text(x = year_start, y = value , s=i, va='top', size=16)
    

# Second plot with data on the last decade with the closer look at the renewable and lower carbon sources exclusively.

rlcsources = ['Biofuel', 'Hydro', 'Nuclear', 'Solar', 'Wind']

we1=we[(we['year'] >= 2015)&(we['year']<=2020)]  
we1.rename(columns = {'biofuel_share_energy': 'Biofuel', 'coal_share_energy': 'Coal', 'gas_share_energy': 'Gas', 'hydro_share_energy' : 'Hydro', 'nuclear_share_energy' : 'Nuclear', 'oil_share_energy' : 'Oil', 'solar_share_energy' :'Solar', 'wind_share_energy': 'Wind'}, inplace=True)

we1= we1[we1['Energy source'].isin(rlcsources)]

for j in rlcsources:
    axes[1].plot(we1[we1['Energy source'] == j]['year'], 
            we1[we1['Energy source'] == j]['Share of primary energy consumption'], 
            alpha=0.7, 
            linewidth=1)


year_start1, year_end = 2015, 2020
names = we1[['Energy source', 'Share of primary energy consumption']][we1['year'] == year_end -1]

for i, value in names.itertuples(index=False):
    axes[1].text(x=year_start1, y=value, s=i, va="top", size=16)
    
#General settings

axes[0].set_title('Share of primary energy consumption 1990-2020', size=18)
axes[1].set_title('Closer look at the last decade', size=18)
axes[0].spines[['top','right','left']].set_visible(False)
axes[1].spines[['top','right','left']].set_visible(False)

axes[0].set_xticks(np.arange(year_start, year_end +1, 5))
axes[0].set_xlim([None, year_end + 0.1])

axes[0].legend(sources, loc='upper right', fontsize='x-large')
axes[1].legend(rlcsources, loc='upper right', fontsize='x-large')

axes[0].text(x=0.1, y=0.99, s="World Energy Mix", transform=fig.transFigure, ha='left', fontsize=20, weight='bold', alpha=.8)
axes[0].add_patch(plt.Rectangle((0.1,0.075),              
                           0.8,                     
                           0.002,                      
                           facecolor='grey', 
                           transform=fig.transFigure, 
                           clip_on=False, 
                           linewidth = 0))

axes[0].add_patch(plt.Rectangle((0.1,0.975),              
                           0.8,                     
                           0.002,                      
                           facecolor='grey', 
                           transform=fig.transFigure, 
                           clip_on=False, 
                           linewidth = 0))

axes[0].text(x=0.1, y=-0.04, s=" • With the increase on fossil fuels (oil, gas and coal) share in overall consumption starting from 1990, byt trend is changing trend over last decade.\n • Among renewable and low carbon sources, hydro, wind and solar energy sources prevails with the recent noticable increase in nuclear energy consumption. \n • At the same time, in different regions of the world share of different energy sources varies significantly.", 
             transform=fig.transFigure, 
             ha='left', 
             fontsize=18, alpha=.8)

axes[0].xaxis.set_tick_params(labelsize=14) 
axes[1].xaxis.set_tick_params(labelsize=14) 
axes[0].yaxis.set_tick_params(labelsize=14) 
axes[1].yaxis.set_tick_params(labelsize=14) 

plt.show()

### Animation: Historical Changes of World Energy Consumption Mix

In [16]:
import plotly.express as px

# Focusing on columns with data on annual percentage change in energy sources consumption starting year 1990
consumption=d.copy()
consumption=consumption.iloc[:, [1, 2, 9, 13, 20, 43, 51, 60, 69, 75, 82, 108, 106, 92, 98, 114, 102, 41-42]]
consumption=consumption[consumption['year']>=1990]

regions=['World', 'Europe', 'Europe (other)','North America', 'Central America',  'South & Central America', 'Other South America','Asia Pacific', 'Other Middle East', 'Other Asia & Pacific',  'CIS', 'Other CIS', 'South Africa', 'Eastern Africa', 
 'Other Northern Africa', 'Other Southern Africa','Other Caribbean']

consumption=consumption[consumption['country'].isin(regions)]

fig = px.bar(consumption, 
             x='country', y='energy_cons_change_pct', 
             color='country',
             animation_frame='year', 
             animation_group="country", 
             range_y=[-20, 20], 
             labels={'energy_cons_change_pct': 'Energy consumption change, %'}, 
             title='Regional Changes in Energy Consumption from 1990, %')

fig.add_vrect(x0=16.5, x1=15.5)
fig.show()

### Animation: Historical World Energy consumption per capita

In [17]:
# Now, focusing on per capita energy consumption patterns to visualize patterns related to the changes in GDP and energy mix consumption 

regions=['Europe', 'Europe (other)','North America', 'Central America',  'South & Central America', 'Other South America','Asia Pacific', 'Other Middle East', 'Other Asia & Pacific',  'CIS', 'Other CIS', 'South Africa', 'Eastern Africa', 
 'Other Northern Africa', 'Other Southern Africa','Other Caribbean']

wemc=d.copy()
wemc=wemc[(wemc['year']>=1990)]
wemc=wemc.loc[wemc['country'].isin(regions)]
wemc=wemc.iloc[:, [1, 2, 42, 47, 98, 99, 122]]

px.scatter(wemc, x='population', y='primary_energy_consumption', 
           animation_frame="year", 
           animation_group="country",
           size='gdp_per_capita', 
           color="country", 
           hover_name="country",      
           size_max=55, 
           log_x=True,
           range_y=[-100, 100000],
           title='Historical changes in primary energy consumption vs GDP per capita growth <br><sup>Asia-Pacific with the largest GDP per capita is significantly increasing the primary energy consumption</sup>', 
           labels={'energy_per_capita': 'Primary yearly energy consumption per capita, kw/hrs', 'gdp_per_capita': 'GDP per capita'})

From the plot above we can see that the highest GDP level in Asia-Pacific also correlated with the increase in primarly energy consumption throughout the years.

<a id="subsection-two"></a>
## 2.2 Energy Consumption profile of countries with the highest GHG emissions

### Global historical GHG emissions

In [18]:
# Let's upload GHG emissions data from Climate Watch used in the previous part of the project and convert wide data (years) to long:

ghg=pd.read_csv('../input/climate-watch-historical-country-ghg-emissions/ghg-emissions.csv')
ghg.isna().sum()

In [19]:
import plotly.express as px


ghg=pd.melt(ghg, id_vars='Country_Region', 
            value_vars=['1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'], 
            value_name='ghg', 
            var_name='year', 
            ignore_index=False)


topghg=ghg.groupby('Country_Region', as_index=False).agg({'ghg': 'sum'}).sort_values('ghg', ascending=False).head(20)
topghg.sort_values('ghg', ascending=False)

# Crating a list of top 20 countries with the highest acumulative GHG emissions

t=[]
for c in topghg['Country_Region']:
    if c not in t:
        t.append(c)

ct=ghg[ghg['Country_Region'].isin(t)]

fig = px.bar(ct, x='Country_Region', y='ghg', color='Country_Region',
             animation_frame="year", 
             animation_group="Country_Region", 
             range_y=[0, 13000], 
             title=('Total GHG emissions, CO2e: top 10 emitters<br><sup>Asia-Pacific and the USA are leading the list with the highest GHG emissions levels</sup>'),
             labels={'ghg': 'Greenhouse gases emissions, ', 'gdp_per_capita': 'GDP per capita', 'Country_Region': 'Region'})
                
fig.show()

China
United States
India
Indonesia
Russia
Brazil
Japan
Iran
Canada
Saudi Arabia
Germany
Democratic Republic of the Congo
Mexico
South Korea
Australia
South Africa
Turkey
Pakistan
Vietnam
Thailand
United Kingdom
Argentina
Malaysia
Italy
Nigeria
France
Egypt
Iraq
Poland
Venezuela
Spain
Algeria
Kazakhstan
Colombia
United Arab Emirates
Myanmar
Bangladesh
Philippines
Ukraine
Peru
Uzbekistan
Ethiopia
Netherlands
Turkmenistan
Tanzania
Bolivia
Kuwait
Angola
Sudan
Libya
Cameroon
Zimbabwe
Qatar
Czech Republic
Belgium
Mozambique
Chad
Oman
Ecuador
Paraguay
Zambia
Morocco
Israel
North Korea
Greece
Romania
Kenya
New Zealand
Cambodia
Austria
Singapore
Belarus
Papua New Guinea
Hungary
Serbia
Portugal
South Sudan
Mongolia
Uganda
Ireland
Finland
Burkina Faso
Chile
Bahrain
Azerbaijan
Botswana
Côte d'Ivoire
Syria
Nepal
Central African Republic
Solomon Islands
Switzerland
Mali
Denmark
Niger
Somalia
Guinea
Madagascar
Dominican Republic
Laos
Guatemala
Nicaragua
Cuba
Sri Lanka
Tunisia
Slovakia
Jordan
Lebanon
Uruguay
Senegal
Republic of Congo
Sweden
Afghanistan
Trinidad and Tobago
Honduras
Norway
Benin
Panama
Yemen
Bosnia and Herzegovina
Namibia
Guyana
Gabon
Malawi
Lithuania
Croatia
Georgia
Bulgaria
Tajikistan
Slovenia
Liberia
Equatorial Guinea
Estonia
El Salvador
Suriname
Kyrgyzstan
Moldova
Mauritania
Ghana
Macedonia
Haiti
Luxembourg
Jamaica
Armenia
Brunei
Sierra Leone
Latvia
Albania
Togo
Costa Rica
Cyprus
Burundi
Rwanda
Belize
Mauritius
Eritrea
Timor-Leste
Guinea-Bissau
Montenegro
Barbados
Bahamas
Gambia
Iceland
Eswatini
Maldives
Lesotho
Grenada
Malta
Djibouti
Antigua and Barbuda
Vanuatu
Samoa
Seychelles
Cape Verde
Saint Lucia
Comoros
Andorra
Sao Tome and Principe
Bhutan
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Tonga
Palau
Marshall Islands
Micronesia
Dominica
Liechtenstein
Kiribati
Cook Islands
Nauru
Tuvalu
Niue
Fiji

In [20]:
import plotly.express as px
consumptionpersource=d.copy()
consumptionpersource=consumptionpersource.iloc[:, [1, 2, 17, 25, 56, 65, 73, 80, 87, 96, 106, 113, 121]]
consumptionpersource=consumptionpersource.fillna(method='ffill').fillna(method='bfill')
consumptionpersource=consumptionpersource[consumptionpersource['year']>=2000]
consumptionpersource['year']=pd.to_datetime(consumptionpersource['year'], format='%Y')
consumptionpersource['year']=consumptionpersource['year'].dt.year
consumptionpersource=consumptionpersource[consumptionpersource['country'].isin(t)]

fig = px.bar(consumptionpersource, 
             x="country", y=['other_renewables_energy_per_capita', 'oil_energy_per_capita', 'nuclear_energy_per_capita', 'hydro_energy_per_capita',
                             'gas_energy_per_capita','coal_cons_per_capita', 'biofuel_cons_per_capita','solar_energy_per_capita', 'wind_energy_per_capita'], 
             title="Consumption Profiles per capita, top emitters",
             color_discrete_map={
                'coal_cons_per_capita':'black',
                'gas_energy_per_capita': '#F0F8FF',
                'oil_energy_per_capita': "#FFB90F",
                'nuclear_energy_per_capita': "#E0EEEE",
                'hydro_energy_per_capita': "#00FFFF",
                'biofuel_cons_per_capita': '#228B22',
                'solar_energy_per_capita': '#FFD700', 
                'wind_energy_per_capita':'#F8F8FF',
                'other_renewables_energy_per_capita': "magenta"}, 
             animation_frame="year", 
             animation_group="country",
                         
             range_y=[0, 200000])
fig.show()

<a id="subsection-three"></a>
### 2.3 Energy Consumption profiles for countries with top performing Smart Cities: focus on Renewables and Low Carbon sources

In [21]:
sci

In [ ]:
# Importing list of Smart Cities from the dataset used in part 2 of the project to join with World Energy Consumption data. The smart cities has been ranked with accordance to their performace measured by IMD Smart Cities Index methodology with the 1st rank being the best.
# We need to check if spellings of countries' names are the same in both lists by comparing them:

sci=pd.read_csv('../input/imdsmartcityindex2020/IMD Smart City Index 2020 report.csv')
sci=sci.iloc[:, [0,1, 6]]

countries=[]

for country in sci['Country']:
    if country not in countries: 
        countries.append(country)

#Preparing a list of countries from the World Energy dataset to compate countries names in both
        
d.groupby('country')

countriesall=[]
for country in d['country']:
    if country not in countriesall: 
        countriesall.append(country)

#Here is the list of countries missing from the IMD Smart City countries' list in the full list of countries due to difference in spelling, so further will replace names of those.

comparison=[]
for i in countries:
    if i not in countriesall:
        comparison.append(i)
comparison 

In [ ]:
# Now replacing the names in the Smart Cities dataset to those in the World Energy one so that we could now select countries from both datasets :

sci['Country'].replace('UAE', 'United Arab Emirates', inplace=True)
sci['Country'].replace('The Netherlands', 'Netherlands', inplace=True)
sci['Country'].replace('UK', 'United Kingdom', inplace=True)
sci['Country'].replace('US', 'United States', inplace=True)
sci['Country'].replace('Singapore ', 'Singapore', inplace=True)
sci['Country'].replace('Isreal', 'Israel', inplace=True)   
temp = sci.sort_values('Ranking', ascending=True)

# Let's add the first 15 countries iterating through the list of Rankings for Smart Cities and add to the final list of countries with the best performing cities only those that are not in the list.

topsmartcountries=[]
x=0
for country in temp.Country:
    if country not in topsmartcountries and x<15:
        x+=1
        topsmartcountries.append(country)
topsmartcountries

In [ ]:
sc=d.copy()
sc=sc.loc[sc['country'].isin(topsmartcountries)]
sc=sc.iloc[:, [1, 2,  73, 96, 98, 114, 106,  121]]
sc=sc[sc['year']>=2000]

#Calculation of total renewable and low carbon energy consumption per capita

sc['renewlowcarbon']=sc['low_carbon_energy_per_capita']+sc['other_renewables_energy_per_capita']+sc['renewables_energy_per_capita']
sc['gdp_per_capita']=sc['gdp']/sc['population']

px.scatter(sc, x='renewlowcarbon', y='gdp_per_capita', 
           animation_frame="year", animation_group="country",
           size='population', 
           color="country", 
           hover_name="country",
           log_x=True, 
           size_max=55, 
           title='Growth of renewables consumption in countries with Smart Cities<br><sup>Noticable growth in renewables and low carbon energy consumption throughout the last decade in countries with the best performing Smart Cities</sup>',
           labels={'renewlowcarbon': 'Consumption of renewable and low carbon energy, per capita', 'gdp_per_capita': 'GDP per capita'})

In [ ]:
# Let's look closer at the Renewable/ow Carbon Energy mix profile for those countries all together

scr=d.copy()
scr=scr.iloc[:, [1, 2, 17, 25, 56, 65, 73, 80, 87, 96, 106, 113, 121]]
scr.fillna(method='ffill').fillna(method='bfill')
scr=scr[scr['year']>2015]

scr=scr[scr['country'].isin(topsmartcountries)]
plot= scr.groupby('country', as_index=False).mean()
plot.plot(x='country', y=[
                'nuclear_energy_per_capita',
                'hydro_energy_per_capita',
                'biofuel_cons_per_capita',
                'solar_energy_per_capita', 
                'wind_energy_per_capita',
                'other_renewables_energy_per_capita'], 
          kind='bar', 
          figsize=(20, 10), 
          title='Renewable/Low Carbon Energy Profile for countries with the top Smart Cities', 
          color={'coal_cons_per_capita':'black',
                'gas_energy_per_capita': '#F0F8FF',
                'oil_energy_per_capita': "#FFB90F",
                'nuclear_energy_per_capita': "grey",
                'hydro_energy_per_capita': "#00FFFF",
                'biofuel_cons_per_capita': '#228B22',
                'solar_energy_per_capita': '#FFD700', 
                'wind_energy_per_capita':'#F8F8FF',
                'other_renewables_energy_per_capita': "magenta"}, width=1)

In [ ]:
fig = px.bar(scr, x="country",
             y=['other_renewables_energy_per_capita', 'oil_energy_per_capita', 'nuclear_energy_per_capita', 'hydro_energy_per_capita','gas_energy_per_capita','coal_cons_per_capita', 'biofuel_cons_per_capita','renewables_energy_per_capita', 'solar_energy_per_capita', 'wind_energy_per_capita'], 
             title="Historical perspective on Renewables and Low Carbon Consumption Profiles <br><sup>Countries with top Smart Cities performers, per capita</sup>",
            color_discrete_map={
                'coal_cons_per_capita':'black',
                'gas_energy_per_capita': '#F0F8FF',
                'oil_energy_per_capita': "#FFB90F",
                'nuclear_energy_per_capita': "#E0EEEE",
                'hydro_energy_per_capita': "#00FFFF",
                'biofuel_cons_per_capita': '#228B22',
                'solar_energy_per_capita': '#FFD700', 
                'wind_energy_per_capita':'#F8F8FF',
                'renewables_energy_per_capita':'#CAFF70',
                'other_renewables_energy_per_capita': "magenta"}, 
             animation_frame="year", 
             animation_group="country",
            range_y=[0, 200000])
fig.show()

<a id="section-three"></a>
# 3. Energy Production patterns

<a id="subsection-one-one"></a>
## 3.1 Fossil fuels energy production profiles for countries with the highest GHG emissions levels (per capita) from 1990

In [ ]:
# Unfortunately, the datasets contains information about energy production for mainly fossil fules (oil, gas and coal).

p=d.copy()

p=p.iloc[:, [1, 2, 27, 58, 89]]
p=p[p['year']>=1990]
p=p.loc[p['country'].isin(t)]

p['ffprod']=p['coal_prod_per_capita']+p['gas_prod_per_capita']+p['oil_prod_per_capita']

import plotly.express as px

fig = px.bar(p, x="country", y="ffprod", color="country",
  animation_frame="year", animation_group="country", 
             range_y=[0, 320000], 
             labels={"ffprod": "Fossil fuels production per capita"}, 
             title='Fossil Fuels Energy Production in top countries with highest GHG emissions')

fig.show()

In [ ]:
# Unfortunately, the datasets contains information about energy production for mainly fossil fules (oil, gas and coal).

p=d.copy()

p=p.iloc[:, [1, 2, 27, 58, 89]]
p=p[p['year']>=1990]
p=p.loc[p['country'].isin(t)]

p['ffprod']=p['coal_prod_per_capita']+p['gas_prod_per_capita']+p['oil_prod_per_capita']

import plotly.express as px

fig = px.bar(p, x="country", y="ffprod", color="country",
  animation_frame="year", animation_group="country", 
             range_y=[0, 320000], 
             labels={"ffprod": "Fossil fuels production per capita"}, 
             title='Fossil Fuels Energy Production in top countries with highest GHG emissions')

fig.show()

<a id="subsection-one-two"></a>
## 3.2 Fossil fuels production of countries with top Smart Cities

In [ ]:
# Separating a list of regions from the list of countries


# Setting the timeframe and picking regions 
wec=d[(d['year']>=2010)&(d['year']<=2020)]

wec= wec[wec["country"].isin(topsmartcountries)]
wec = wec.fillna(method="ffill").fillna(method="bfill")

wec['renlowcarbon'] = wec['other_renewables_energy_per_capita']+ wec['renewables_energy_per_capita']+ wec['low_carbon_energy_per_capita']

column='renlowcarbon'

fig, ax = plt.subplots(figsize=(15,20))

sns.set_style(style='white')

ax.grid(which="major", axis='y', color='#758D99', alpha=0.6, zorder=1)

for region in topsmartcountries:
    ax.plot(wec[wec['country'] == region]['year'], 
            wec[wec['country'] == region]['renlowcarbon'], 
            alpha=0.7, 
            linewidth=1)
       
ax.plot(wec[wec['country'] == 'World']['year'], 
        wec[wec['country'] == 'World']['renlowcarbon'], 
        color='grey',
        linewidth=3)

year_start, year_end = 2010, 2020

labels = wec[['country', column]][wec['year'] == year_end-1]

for region, value in labels.itertuples(index=False):
    ax.text(x=year_start, y=value, s=region, va="center")

ax.spines[['top','right','left', 'bottom']].set_visible(False)

ax.set_xticks(np.arange(year_start, year_end +.5, 1))
ax.set_xlim([None, year_end + 1])

# Add in title and subtitle
ax.text(x=0.1, y=0.94, s="Renewables/Low Carbon energy and Smart Cities", transform=fig.transFigure, ha='left', fontsize=20, weight='bold', alpha=.8)
ax.text(x=0.1, y=0.92, s="Consumption in countries with top Smart Cities, per capita", transform=fig.transFigure, ha='left', fontsize=16, alpha=.8)

plt.xlabel('Year') 
ax.xaxis.set_tick_params(labelsize=12) 
plt.ylabel('Renewables/Low Carbon energy, per capita')
   
plt.show()

<a id="section-four"></a>
# Insights:

* Throughout the last decade there is a noticable shift to Renewables/Low Carbon energy sources' consumption worldwide and specifically in coutries with top performing Smart Cities.
* Asia-Pacific is the region with the largest gdp per capita and GHG emissions numbers, as well as huge Renewables/Low Carbon energy consumption potential.
* Asia-Pacific and the North America are the largest energy consumers. Among consumers of Renewable/Low carbon energy sources Norway, Canada and Sweden are leading the list.
* Fossil fuels (oil, gas and coal) are stil the primary energy sources however hydro energy and nuclear are the most popular among renewables/low carbon sources.

[Check out the 1st part of this project here.](https://www.kaggle.com/code/olgashevtsova/1-smart-cities-visualizing-sci-with-python?scriptVersionId=100552594)
[Check out the 2nd part of this project here.](https://www.kaggle.com/code/olgashevtsova/2-smart-cities-population-and-smi-raiting/)


Thank you for visiting this page, for any comments and suggestions!
Sincerely,
Luca